<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2024notebooks/2025_0110stable_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# diffuser を用いた **[Stable Diffusion](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb#scrollTo=2rR2Udg5IYjn)**

Stable Diffusion は [CompVis](https://github.com/CompVis), [Stability AI](https://stability.ai/), [LAION](https://laion.ai/) の研究者とエンジニアによって作成されたテキストから画像へ変換する潜在拡散モデルである。
これは [LAION-5B](https://laion.ai/blog/laion-5b/) データベースのサブセットから 512x512 画像で学習される。
このモデルは，凍結された CLIP ViT-L/14 テキスト符号化器を使い，テキストプロンプトを条件としている。
860M の U-Net と 123M のテキスト符号化器を持つこのモデルは，比較的軽量で，多くの民生用 GPU で実行可能である。
詳しくは [モデルカード](https://huggingface.co/CompVis/stable-diffusion) を参照。
<!-- Stable Diffusion is a text-to-image latent diffusion model created by the researchers and engineers from [CompVis](https://github.com/CompVis), [Stability AI](https://stability.ai/) and [LAION](https://laion.ai/).
It's trained on 512x512 images from a subset of the [LAION-5B](https://laion.ai/blog/laion-5b/) database.
This model uses a frozen CLIP ViT-L/14 text encoder to condition the model on text prompts.
With its 860M UNet and 123M text encoder, the model is relatively lightweight and can run on many consumer GPUs.
See the [model card](https://huggingface.co/CompVis/stable-diffusion) for more information. -->


本 Colab ノートブックでは Hugging Face [Diffusers ライブラリ](https://github.com/huggingface/diffusers) を使った Stable Diffusion の使い方を紹介する。
<!-- This Colab notebook shows how to use Stable Diffusion with the 🤗 Hugging Face [🧨 Diffusers library](https://github.com/huggingface/diffusers).

さっそく始めましょう！<!--Let's get started! -->

## 1. `StableDiffusionPipline` の使い方
<!-- ## 1. How to use `StableDiffusionPipeline` -->

Stable Diffusion がどのように機能するか理論的な側面に飛び込む前に，少し試してみよう。
<!-- Before diving into the theoretical aspects of how Stable Diffusion functions, let's try it out a bit 🤗.-->

本節では，わずか数行のコードでテキストから画像への推論を実行する方法を紹介する！
<!--In this section, we show how you can run text to image inference in just a few lines of code! -->

### セットアップ
<!-- ### Setup-->

最初に、このノートブックを実行するために GPU ランタイムを使用していることを確認せよ。
以下のコマンドが失敗する場合は，上の  `Runtime`  メニューから `Change runtime type`  を選択せよ。
<!--First, please make sure you are using a GPU runtime to run this notebook, so inference is much faster.
If the following command fails, use the `Runtime` menu above and select `Change runtime type`. -->

In [1]:
import torch

In [2]:
if torch.cuda.is_available():
    # Shows the nVidia GPUs, if this system has any
    !nvidia-smi

Fri Jul 18 00:51:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P8             11W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

次に `diffusers`  と `scipy`, `ftfy`, `transformers` をインストールする。
`accelerate` は読み込みを高速化するために使用する。
<!-- Next, you should install `diffusers` as well `scipy`, `ftfy` and `transformers`. `accelerate` is used to achieve much faster loading. -->

In [ ]:
!pip install diffusers==0.11.1
!pip install transformers scipy ftfy accelerate

### Stable Diffusion パイプライン
<!-- ### Stable Diffusion Pipeline-->

`StableDiffusionPipeline` は，わずか数行のコードでテキストから画像を生成するために使用できるエンドツーエンドの推論パイプラインである。
<!-- `StableDiffusionPipeline` is an end-to-end inference pipeline that you can use to generate images from text with just a few lines of code. -->

まず，モデルの全成分の事前学習された重みをロードする。
本ノートブックでは Stable Diffusion version 1.4 ([CompVis/stable-diffusion-v1-4](https://huggingface.co/CompVis/stable-diffusion-v1-4)) を使うが，他にも試してみたいものがあるかもしれない：
<!--First, we load the pre-trained weights of all components of the model. In this notebook we use Stable Diffusion version 1.4 ([CompVis/stable-diffusion-v1-4](https://huggingface.co/CompVis/stable-diffusion-v1-4)), but there are other variants that you may want to try: -->
* [runwayml/stable-diffusion-v1-5](https://huggingface.co/runwayml/stable-diffusion-v1-5)
* [stabilityai/stable-diffusion-2-1-base](https://huggingface.co/stabilityai/stable-diffusion-2-1-base)
* [stabilityai/stable-diffusion-2-1](https://huggingface.co/stabilityai/stable-diffusion-2-1).
このバージョンは 768x768 の解像度の画像を作成できるが、他のバージョンは 512x512 で動作する。
<!-- This version can produce images with a resolution of 768x768, while the others work at 512x512. -->

モデル ID [CompVis/stable-diffusion-v1-4](https://huggingface.co/CompVis/stable-diffusion-v1-4) に加えて、特定の `revision` と `torch_dtype` を `from_pretrained` メソッドに渡している。
<!-- In addition to the model id [CompVis/stable-diffusion-v1-4](https://huggingface.co/CompVis/stable-diffusion-v1-4), we're also passing a specific `revision` and `torch_dtype` to the `from_pretrained` method.-->


フリーの Google Colab がすべて Stable Diffusion を実行できるようにしたいので、半精度ブランチ[`fp16`](https://huggingface.co/CompVis/stable-diffusion-v1-4/tree/fp16) から重みをロードし、`torch_dtype=torch.float16` を渡すことで `diffusers` に float16 の精度で重みを期待するように伝えている。
<!-- We want to ensure that every free Google Colab can run Stable Diffusion, hence we're loading the weights from the half-precision branch [`fp16`](https://huggingface.co/CompVis/stable-diffusion-v1-4/tree/fp16) and also tell `diffusers` to expect the weights in float16 precision by passing `torch_dtype=torch.float16`. -->

可能な限り高い精度を確保したい場合は、メモリ使用量が増えることを覚悟で `torch_dtype=torch.float16` を削除してください。
<!-- If you want to ensure the highest possible precision, please make sure to remove `torch_dtype=torch.float16` at the cost of a higher memory usage. -->

In [ ]:
# This is added to get around some issues of Torch not loading models correctly (test on Mac OS X and Kubuntu Linux)
!pip install --upgrade huggingface-hub==0.26.2 transformers==4.46.1 tokenizers==0.20.1 diffusers==0.31.0

In [ ]:
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)

より高速な推論を行うため，パイプラインを GPU に移す。
<!-- Next, let's move the pipeline to GPU to have faster inference. -->

In [5]:
import torch
if torch.cuda.is_available():
    device=torch.device("cuda")
elif torch.backends.mps.is_available():
    device=torch.device("mps")

pipe = pipe.to(device)

画像を生成する準備が整った：
<!-- And we are ready to generate images: -->

In [ ]:
prompt = "a photograph of an astronaut riding a horse"
image = pipe(prompt).images[0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)

# Now to display an image you can either save it such as:
image.save(f"astronaut_rides_horse.png")

# or if you're in a google colab you can directly display it with
image

上記のセルを複数回実行すると，毎回異なる画像が得られる。
もし決定論的な出力を望むなら，パイプラインにランダムな種を渡すことができる。
同じ種を使うたびに，同じ画像結果が得られる。
<!-- Running the above cell multiple times will give you a different image every time.
If you want deterministic output you can pass a random seed to the pipeline.
Every time you use the same seed you'll have the same image result. -->

In [ ]:
seed=42
generator = torch.Generator(device).manual_seed(seed)
image = pipe(prompt, generator=generator).images[0]
image

`num_inference_steps` 引数で推論ステップ数を変更できる。
一般的に，ステップ数が多いほど良い結果が得られる。
最新のモデルの 1 つである Stable Diffusion は，比較的少ないステップ数でうまく機能するので，デフォルトの`50` を使用することを推奨する。
より速い結果を得たいのであれば，より小さい数を使うことができる。
<!-- You can change the number of inference steps using the `num_inference_steps` argument.
In general, results are better the more steps you use.
Stable Diffusion, being one of the latest models, works great with a relatively small number of steps, so we recommend to use the default of `50`.
If you want faster results you can use a smaller number.-->


次のセルは先ほどと同じ種を使用しているが，ステップ数は少なくなっている。
馬の頭やヘルメットのような細部は，前の画像よりもリアルではっきりしていないことに注意：
<!--The following cell uses the same seed as before, but with fewer steps.
Note how some details, such as the horse's head or the helmet, are less defin realistic and less defined than in the previous image: -->

In [ ]:
seed=42

steps=50
steps=15
generator = torch.Generator(device).manual_seed(seed)
image = pipe(prompt, num_inference_steps=steps, generator=generator).images[0]
image

パイプラインコールのもう1つのパラメータは `guidance_scale` である。
これは、条件信号（この場合はテキスト）、および全体的なサンプルの品質への準拠を高める方法である。
簡単に言うと、分類器を使わないガイダンスは、よりプロンプトにマッチするように生成を強制する。
`7` や `8.5` のような数字は良い結果をもたらすが，非常に大きな数字を使用した場合，画像は良く見えるかもしれないが，多様性に欠ける。
<!-- The other parameter in the pipeline call is `guidance_scale`.
It is a way to increase the adherence to the conditional signal which in this case is text as well as overall sample quality.
In simple terms classifier free guidance forces the generation to better match with the prompt.
Numbers like `7` or `8.5` give good results, if you use a very large number the images might look good, but will be less diverse. -->


このパラメータの技術的な詳細については，このノート の[最終節](https://colab.research.google.com/drive/1ALXuCM5iNnJDNW5vqBm5lCtUQtZJHN2f?authuser=1#scrollTo=UZp-ynZLrS-S) を参照。
<!-- You can learn about the technical details of this parameter in [the last section](https://colab.research.google.com/drive/1ALXuCM5iNnJDNW5vqBm5lCtUQtZJHN2f?authuser=1#scrollTo=UZp-ynZLrS-S) of this notebook. -->

同じプロンプトに対して複数の画像を生成するには、同じプロンプトを複数回繰り返したリストを使用する。
先ほど使った文字列の代わりに，このリストをパイプラインに送る。
<!-- To generate multiple images for the same prompt, we simply use a list with the same prompt repeated several times.
We'll send the list to the pipeline instead of the string we used before. -->

まずは画像のグリッドを表示するヘルパー関数を書いてみよう。
以下のセルを実行するだけで、`image_grid`関数が作成される。
<!-- Let's first write a helper function to display a grid of images.
Just run the following cell to create the `image_grid` function, or disclose the code if you are interested in how it's done. -->

In [9]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

これで、3 つのプロンプトのリストでパイプラインを実行すれば、グリッド画像を生成できる。
<!-- Now, we can generate a grid image once having run the pipeline with a list of 3 prompts. -->

In [ ]:
num_images = 3
prompt = ["a photograph of an astronaut riding a horse"] * num_images

images = pipe(prompt).images

grid = image_grid(images, rows=1, cols=3)
grid

次に `n × m` の画像のグリッドを生成する方法
<!-- And here's how to generate a grid of `n × m` images. -->

In [ ]:
num_cols = 3
num_rows = 4

prompt = ["a photograph of an astronaut riding a horse"] * num_cols

all_images = []
for i in range(num_rows):
  images = pipe(prompt).images
  all_images.extend(images)

grid = image_grid(all_images, rows=num_rows, cols=num_cols)
grid

### 正方形以外の画像を生成
<!-- ### Generate non-square images-->

Stable Diffusion はデフォルトで `512 × 512` 画素の画像を生成する。
しかし、`height` と `width` 引数を使えば簡単にデフォルトを上書きできるので、縦長や横長の長方形の画像を作成することができる。
<!-- Stable Diffusion produces images of `512 × 512` pixels by default.
But it's very easy to override the default using the `height` and `width` arguments, so you can create rectangular images in portrait or landscape ratios. -->

以下は適切な画像サイズを選ぶための推奨事項：
- `height` と `width` が両方とも `8` の倍数であることを確認せよ
- 512 を下回ると低画質になる可能性がある。
- 両方向で 512 を超えると、画像領域が繰り返されます（全体的な一貫性が失われる）。
- 正方形でない画像を作成する最善の方法は、1 つの次元で `512` を使用し、もう 1つ の次元でそれよりも大きな値を使用することである。

<!--These are some recommendations to choose good image sizes:
- Make sure `height` and `width` are both multiples of `8`.
- Going below 512 might result in lower quality images.
- Going over 512 in both directions will repeat image areas (global coherence is lost).
- The best way to create non-square images is to use `512` in one dimension, and a value larger than that in the other one. -->

In [ ]:
prompt = "a photograph of an astronaut riding a horse"

image = pipe(prompt, height=512, width=768).images[0]
image


## 2. Stable Diffusion とは何か<!--2. What is Stable Diffusion-->

Stable Diffusion の理論的な部分に入ろう。
<!-- Now, let's go into the theoretical part of Stable Diffusion 👩‍🎓.-->

Stable Diffusion は [High-Resolution Image Synthesis with Latent Diffusion Models](https://arxiv.org/abs/2112.10752) で提案された、**Latent Diffusion** と呼ばれる特殊な拡散モデルに基づいている。
<!--Stable Diffusion is based on a particular type of diffusion model called **Latent Diffusion**, proposed in [High-Resolution Image Synthesis with Latent Diffusion Models](https://arxiv.org/abs/2112.10752). -->



一般的な拡散モデルは、ランダムな正規雑音を段階的に **雑音除去** するように訓練された機械学習システムであり、**画像** のような関心のあるサンプルに到達する。
拡散モデルがどのように機能するかのより詳細な概要については、[この研究室](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/diffusers_intro.ipynb) を参照してほしい。
<!-- General diffusion models are machine learning systems that are trained to *denoise* random gaussian noise step by step, to get to a sample of interest, such as an *image*.
For a more detailed overview of how they work, check [this colab](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/diffusers_intro.ipynb).-->

拡散モデルは，画像データの生成において最先端の結果を達成することが示されている。
しかし拡散モデルの欠点は，逆雑音処理に時間がかかることである。
さらに，これらのモデルは画素空間で動作するため多くのメモリを消費し，高解像度の画像を生成する場合には不当に高価になる。
したがって，これらのモデルを学習し，推論に使用することは困難である。
<!-- Diffusion models have shown to achieve state-of-the-art results for generating image data.
But one downside of diffusion models is that the reverse denoising process is slow.
In addition, these models consume a lot of memory because they operate in pixel space, which becomes unreasonably expensive when generating high-resolution images.
Therefore, it is challenging to train these models and also use them for inference. -->

<br>

潜在拡散モデルは、実際の画素空間を使用する代わりに、低次元の __潜在__空間上で拡散処理を適用することにより，メモリと計算の複雑さを軽減することができる。
これが標準的な拡散と潜在拡散モデルの主な違いです： **潜在拡散モデル** では、モデルは画像の潜在的な（圧縮された）表現を生成するように学習される。
<!-- Latent diffusion can reduce the memory and compute complexity by applying the diffusion process over a lower dimensional _latent_ space, instead of using the actual pixel space.
This is the key difference between standard diffusion and latent diffusion models: **in latent diffusion the model is trained to generate latent (compressed) representations of the images.** -->

 潜在拡散モデルには 3 つの主要な成分がある。
<!--There are three main components in latent diffusion. -->

1. 自己符号化器 (VAE)
2. A [U-Net](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/diffusers_intro.ipynb#scrollTo=wW8o1Wp0zRkq).
3. テキスト符号化器，[CLIP's テキスト符号化器](https://huggingface.co/docs/transformers/model_doc/clip#transformers.CLIPTextModel).

<!-- 1. An autoencoder (VAE).
2. A [U-Net](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/diffusers_intro.ipynb#scrollTo=wW8o1Wp0zRkq).
3. A text-encoder, *e.g.* [CLIP's Text Encoder](https://huggingface.co/docs/transformers/model_doc/clip#transformers.CLIPTextModel). -->

**1. 自己符号化器 VAE**
<!-- **1. The autoencoder (VAE)**-->

VAE モデルには，符号化器と復号化器の 2 つの部分がある。
復号化器は画像を低次元の潜在表現に変換するために使用され，これは **U-Net** モデルの入力となる。
復号化器は逆に，潜在表現を画像に変換する。
<!-- The VAE model has two parts, an encoder and a decoder. The encoder is used to convert the image into a low dimensional latent representation, which will serve as the input to the *U-Net* model.
The decoder, conversely, transforms the latent representation back into an image. -->

潜像拡散モデルの「訓練」の間，符号化器は順方向拡散処理のために画像の潜像表現 (_latents_) を得るために使われ，各段階でより多くの雑音が適用される。
推論中は，逆拡散処理で生成された雑音除去された潜在変数は，VAE 復号化器を使って画像に変換される。
後述するように，推論中に必要なのは **VAE 復号化器** だけである。
<!--During latent diffusion _training_, the encoder is used to get the latent representations (_latents_) of the images for the forward diffusion process, which applies more and more noise at each step. During _inference_, the denoised latents generated by the reverse diffusion process are converted back into images using the VAE decoder. As we will see during inference we **only need the VAE decoder**. -->

**2. Uネット**<!-- **2. The U-Net**-->

U-Net は ResNet ブロックで構成された符号化器部と復号化器ダ部を持つ。
符号化器は画像表現を低解像度の画像表現に圧縮し，復号化器は低解像度の画像表現を雑音の少ない元の高解像度の画像表現に復号化する。
より具体的には，U-Net の出力は雑音残差を予測し，これを用いて予測された雑音除去画像表現を計算することができる。
<!-- The U-Net has an encoder part and a decoder part both comprised of ResNet blocks.
The encoder compresses an image representation into a lower resolution image representation and the decoder decodes the lower resolution image representation back to the original higher resolution image representation that is supposedly less noisy.
More specifically, the U-Net output predicts the noise residual which can be used to compute the predicted denoised image representation. -->

U-Net がダウンサンプリング中に重要な情報を失うのを防ぐため，通常，符号化器のダウンサンプリング ResNets と復号化器のアップサンプリングResNets の間にショートカット接続が追加される。
さらに，Stable diffusion U-Net は交差注意層を介して，その出力をテキスト埋め込みに条件付けることができる。
交差注意層は U-Net 符号化器部と復号化器部の両方に追加され，通常は ResNet ブロックの間に配置される。
<!--To prevent the U-Net from losing important information while downsampling, short-cut connections are usually added between the downsampling ResNets of the encoder to the upsampling ResNets of the decoder.
Additionally, the stable diffusion U-Net is able to condition its output on text-embeddings via cross-attention layers.
The cross-attention layers are added to both the encoder and decoder part of the U-Net usually between ResNet blocks. -->

**3. テキスト・エンコーダ**
<!-- **3. The Text-encoder**-->

テキストエンコーダは、入力されたプロンプト、例えば*「An astronout riding a horse」（馬に乗った宇宙飛行士）を、U-Netが理解できる埋め込み空間に変換する役割を担っています。
それは通常、入力トークンのシーケンスを潜在的なテキスト埋め込みシーケンスにマッピングする単純な*変換器ベース*エンコーダです。
<!-- The text-encoder is responsible for transforming the input prompt, *e.g.* "An astronout riding a horse" into an embedding space that can be understood by the U-Net.
It is usually a simple *transformer-based* encoder that maps a sequence of input tokens to a sequence of latent text-embeddings. -->


[Imagen](https://imagen.research.google/) に触発された Stable Diffusion モデルは，学習中にテキスト符号化器の学習を行わず，単に CLIP の学習済みのテキスト符号化器である [CLIP テキストモデル](https://huggingface.co/docs/transformers/model_doc/clip#transformers.CLIPTextModel) を使用する。
<!--Inspired by [Imagen](https://imagen.research.google/), Stable Diffusion does **not** train the text-encoder during training and simply uses an CLIP's already trained text encoder, [CLIPTextModel](https://huggingface.co/docs/transformers/model_doc/clip#transformers.CLIPTextModel). -->

**なぜ潜在拡散モデルは高速で効率的なのか？**
<!-- **Why is latent diffusion fast and efficient?**-->

潜在拡散モデルの U-Net は低次元空間で動作するため，画素空間の拡散モデルと比較して，メモリと計算要件が大幅に削減される。
例えば Stable Diffusion で使用されている自己符号化器は，削減係数 reduction factor 8を持っている。
つまり `(3, 512, 512)` の形状の画像は，潜在空間では `(3, 64, 64)` となり，必要なメモリは `8 × 8 = 64` 分の 1 になる。
<!-- Since the U-Net of latent diffusion models operates on a low dimensional space, it greatly reduces the memory and compute requirements compared to pixel-space diffusion models.
For example, the autoencoder used in Stable Diffusion has a reduction factor of 8.
This means that an image of shape `(3, 512, 512)` becomes `(3, 64, 64)` in latent space, which requires `8 × 8 = 64` times less memory. -->

これが 16 GBの Colab GPU でも`512 × 512` の画像を高速に生成できる理由である！
<!--This is why it's possible to generate `512 × 512` images so quickly, even on 16GB Colab GPUs! -->

**推論中の安定拡散モデル**
<!-- **Stable Diffusion during inference**-->

では、論理的な流れを説明しながら、推論においてモデルがどのように機能するかを詳しく見ていこう。
<!--Putting it all together, let's now take a closer look at how the model works in inference by illustrating the logical flow. -->


<p align="left">
<img src="https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/stable_diffusion.png" alt="sd-pipeline" width="500"/>
</p>

Stable Diffusion モデルは潜在種とテキストプロンプトを入力とする。
潜在種は，サイズ $64\times 64$ のランダムな潜在的な画像表現を生成するのに使われ、テキストプロンプトはCLIP のテキスト符号化器を介してサイズ $77\times768$ のテキスト埋め込み表現に変換される。
<!-- The stable diffusion model takes both a latent seed and a text prompt as an input.
The latent seed is then used to generate random latent image representations of size $64 \times 64$ where as the text prompt is transformed to text embeddings of size $77 \times 768$ via CLIP's text encoder.-->


次に，U-Net  はテキスト埋め込みを条件としながら，ランダムな潜在画像表現を繰り返し **雑音除去** する。
雑音残差である U-Net の出力は，スケジューラアルゴリズムを介して雑音除去された潜在画像表現を計算するために使われる。
この計算には多くの異なるスケジューラアルゴリズムを用いることができ，それぞれに長所と短所がある。
Stable Diffusion については、以下のいずれかを使用することを推奨する：
<!--Next the U-Net iteratively *denoises* the random latent image representations while being conditioned on the text embeddings.
The output of the U-Net, being the noise residual, is used to compute a denoised latent image representation via a scheduler algorithm.
Many different scheduler algorithms can be used for this computation, each having its pros and cons.
For Stable Diffusion, we recommend using one of: -->

- [PNDM scheduler](https://github.com/huggingface/diffusers/blob/main/src/diffusers/schedulers/scheduling_pndm.py) (used by default).
- [K-LMS scheduler](https://github.com/huggingface/diffusers/blob/main/src/diffusers/schedulers/scheduling_lms_discrete.py).
- [Heun Discrete scheduler](https://github.com/huggingface/diffusers/blob/main/src/diffusers/schedulers/scheduling_heun_discrete.py).
- [DPM Solver Multistep scheduler](https://github.com/huggingface/diffusers/blob/main/src/diffusers/schedulers/scheduling_dpmsolver_multistep.py).
このスケジューラは、少ないステップで素晴らしい品質を達成することができる。
デフォルトの 50 ではなく、25 で試してみて欲しい！
<!-- This scheduler is able to achieve great quality in less steps.
You can try with 25 instead of the default 50! -->


スケジューラアルゴリズムがどのように機能するかについての理論は，ここでは扱わないが，要するにスケジューラは，以前の雑音表現と予測された雑音残差から，予測された雑音除去画像表現を計算することを覚えておく必要がある。
詳しくは [拡散に基づく生成モデルの設計空間の解明](https://arxiv.org/abs/2206.00364) を参照することをお勧めする。
<!-- Theory on how the scheduler algorithm function is out of scope for this notebook, but in short one should remember that they compute the predicted denoised image representation from the previous noise representation and the predicted noise residual.
For more information, we recommend looking into [Elucidating the Design Space of Diffusion-Based Generative Models](https://arxiv.org/abs/2206.00364)-->


この  **雑音除去** 処理は，より良い潜像表現を段階的に取り出すために **約50回** 繰り返される。
一旦完了すると，潜在画像表現は変分自己符号化器の復号化器部によって復号化される。
<!--The *denoising* process is repeated *ca.* 50 times to step-by-step retrieve better latent image representations.
Once complete, the latent image representation is decoded by the decoder part of the variational auto encoder. -->


潜在変数と stable diffusion について簡単に紹介した後 Huggingface の diffusers の高度な使い方を見てみよう！
<!-- After this brief introduction to Latent and Stable Diffusion, let's see how to make advanced use of 🤗 Hugging Face Diffusers! -->

## 3. `diffusers` を使って独自の推論パイプラインを書く方法<!-- ## 3. How to write your own inference pipeline with `diffusers`-->

最後に、`diffusers` を使ってカスタムの推論パイプラインを作成する方法を紹介する。
これは系の特定の機能をもう少し深く掘り下げたり，特定の成分を切り替えたりするのに非常に便利である。
<!-- Finally, we show how you can create custom diffusion pipelines with `diffusers`.
This is often very useful to dig a bit deeper into certain functionalities of the system and to potentially switch out certain components. -->

本節では、異なるスケジューラ、すなわち [このPR](https://github.com/huggingface/diffusers/pull/185#pullrequestreview-1074247365)で追加された[Katherine Crowson](https://github.com/crowsonkb) の K-LMS スケジューラで Stable Diffusion を使用する方法を示す。
<!--In this section, we will demonstrate how to use Stable Diffusion with a different scheduler, namely [Katherine Crowson's](https://github.com/crowsonkb) K-LMS scheduler that was added in [this PR](https://github.com/huggingface/diffusers/pull/185#pullrequestreview-1074247365). -->

`StableDiffusionPipeline` をそれぞれ見ていこう。
<!-- Let's go through the `StableDiffusionPipeline` step by step to see how we could have written it ourselves.-->

まず，それぞれのモデルをロードすることから始めよう。
<!--We will start by loading the individual models involved. -->

[訓練済モデル](https://huggingface.co/CompVis/stable-diffusion-v1-3-diffusers/tree/main) には，完全な stable  diffusion パイプラインをセットアップするために必要なすべての成分が含まれている。
これらは以下のフォルダに格納されている：
- `text_encoder`  Stable Diffusion は CLIP を使用しているが、他の拡散モデルでは `BERT` のような他の符号化器を使用することもある。
- `tokenizer`:   `text_encoder` モデルで使用されているものと一致しなければならない
-  `schduler `： 訓練中に画像に雑音を徐々に追加するためのスケジューリングアルゴリズム
-` unet`： 入力の潜在表現を生成するためのモデル
- `vae`： 潜在表現を実画像に復号化するために使用する自己符号化器モジュール

`from_pretrained` の引数  `subfolder` を使って，成分を保存したフォルダを参照することで，そこにある成分をロードすることができる。

<!-- The [pre-trained model](https://huggingface.co/CompVis/stable-diffusion-v1-3-diffusers/tree/main) includes all the components required to setup a complete diffusion pipeline. They are stored in the following folders:
- `text_encoder`: Stable Diffusion uses CLIP, but other diffusion models may use other encoders such as `BERT`.
- `tokenizer`. It must match the one used by the `text_encoder` model.
- `scheduler`: The scheduling algorithm used to progressively add noise to the image during training.
- `unet`: The model used to generate the latent representation of the input.
- `vae`: Autoencoder module that we'll use to decode latent representations into real images.

We can load the components by referring to the folder they were saved, using the `subfolder` argument to `from_pretrained`. -->

In [ ]:
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler

# 1. Load the autoencoder model which will be used to decode the latents into image space.
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")

# 2. Load the tokenizer and text encoder to tokenize and encode the text.
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

# 3. The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet")

ここでは，定義済みのスケジューラーをロードする代わりに K-LMS スケジューラーを使うことにする。
<!-- Now instead of loading the pre-defined scheduler, we'll use the K-LMS scheduler instead. -->

In [ ]:
from diffusers import LMSDiscreteScheduler

scheduler = LMSDiscreteScheduler.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="scheduler")

次に、モデルをGPUに移す。
<!-- Next we move the models to the GPU. -->

In [ ]:
vae = vae.to(device)
text_encoder = text_encoder.to(device)
unet = unet.to(device)

ここで，画像生成に使用するパラメータを定義する。
<!-- We now define the parameters we'll use to generate images.-->

`guidance_scale` は [Imagen 論文](https://arxiv.org/pdf/2205.11487.pdf) の式(2) のガイダンスの重み `w` に類似して定義されていることに注意。
`guidance_scale == 1` は分類器を使わないガイダンスを行うことに相当する。ここでは7.5に設定する。
<!-- Note that `guidance_scale` is defined analog to the guidance weight `w` of equation (2) in the [Imagen paper](https://arxiv.org/pdf/2205.11487.pdf). `guidance_scale == 1` corresponds to doing no classifier-free guidance. Here we set it to 7.5 as also done previously. -->

前例とは対照的に，より明確な画像を得るために `num_inference_steps` を 100 に設定する。
<!--In contrast to the previous examples, we set `num_inference_steps` to 100 to get an even more defined image. -->

In [ ]:
prompt = ["a photograph of an astronaut riding a horse"]

height = 512                        # default height of Stable Diffusion
width = 512                         # default width of Stable Diffusion

num_inference_steps = 100            # Number of denoising steps

guidance_scale = 7.5                # Scale for classifier-free guidance

generator = torch.manual_seed(32)   # Seed generator to create the inital latent noise

batch_size = 1

まず，プロンプトの text_embeddings を得る。
これらの埋め込みは UNet モデルの条件に使われる。
<!-- First, we get the text_embeddings for the prompt.
These embeddings will be used to condition the UNet model. -->

In [ ]:
text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")

with torch.no_grad():
  text_embeddings = text_encoder(text_input.input_ids.to(device))[0]

これは 埋草 (パディング)トークン (空のテキスト) の埋め込みである。
これらは条件付きの `text_embeddings` (`batch_size` と `seq_length`) と同じ次元を持つ必要がある。
<!-- We'll also get the unconditional text embeddings for classifier-free guidance, which are just the embeddings for the padding token (empty text).
They need to have the same shape as the conditional `text_embeddings` (`batch_size` and `seq_length`) -->

In [ ]:
max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer(
    [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
)
with torch.no_grad():
  uncond_embeddings = text_encoder(uncond_input.input_ids.to(device))[0]

分類器を使わないガイダンスでは，2 つのフォワードパスを行う必要がある。
一つは条件付き入力（`text_embeddings`）で、もう一つは無条件埋め込み（`uncond_embeddings`）である。
実際には、2 つのフォワードパスを回避するために、両方を 1 つのバッチに連結することができる。
<!-- For classifier-free guidance, we need to do two forward passes.
One with the conditioned input (`text_embeddings`), and another with the unconditional embeddings (`uncond_embeddings`). In practice, we can concatenate both into a single batch to avoid doing two forward passes. -->

In [ ]:
text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

初期ランダムノイズを生成する。
<!-- Generate the intial random noise. -->

In [ ]:
latents = torch.randn(
  (batch_size, unet.in_channels, height // 8, width // 8),
  generator=generator,
)
latents = latents.to(device)

In [ ]:
latents.shape

クールな $64 \times 64$ が期待される。
モデルは，この潜在表現 (純粋なノイズ) を後で `512 × 512` 画像に変換する。
<!-- Cool $64 \times 64$ is expected. The model will transform this latent representation (pure noise) into a `512 × 512` image later on.-->

次に、選択した `num_inference_steps` を使用してスケジューラを初期化する。

これにより，ノイズ除去処理中に使用される `sigmas` と正確な時間ステップ値が計算される。
<!--Next, we initialize the scheduler with our chosen `num_inference_steps`.
This will compute the `sigmas` and exact time step values to be used during the denoising process. -->

In [ ]:
scheduler.set_timesteps(num_inference_steps)

K-LMS スケジューラは、`latents` に `sigma` 値を掛ける必要がある。
ここでこれを行う。
<!-- The K-LMS scheduler needs to multiply the `latents` by its `sigma` values. Let's do this here -->

In [ ]:
latents = latents * scheduler.init_noise_sigma

これでノイズ除去ループを書く準備ができた。
<!-- We are ready to write the denoising loop. -->

In [ ]:
from tqdm.auto import tqdm
from torch import autocast

for t in tqdm(scheduler.timesteps):
  # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
  latent_model_input = torch.cat([latents] * 2)

  latent_model_input = scheduler.scale_model_input(latent_model_input, t)

  # predict the noise residual
  with torch.no_grad():
    noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample

  # perform guidance
  noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
  noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

  # compute the previous noisy sample x_t -> x_t-1
  latents = scheduler.step(noise_pred, t, latents).prev_sample

次に，`vae` を使って，生成された `latents` を画像に復号化する。
<!-- We now use the `vae` to decode the generated `latents` back into the image. -->

In [ ]:
# scale and decode the image latents with vae
latents = 1 / 0.18215 * latents

with torch.no_grad():
  image = vae.decode(latents).sample

そして最後に，画像を PIL に変換して表示または保存できるようにする。
<!-- And finally, let's convert the image to PIL so we can display or save it. -->

In [ ]:
image = (image / 2 + 0.5).clamp(0, 1)
image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
images = (image * 255).round().astype("uint8")
pil_images = [Image.fromarray(image) for image in images]
pil_images[0]

これで，独自のパイプラインを構築したり，diffuser 成分を好きなように使用したりするためのすべてのパーツが揃った。
<!-- Now you have all the pieces to build your own pipelines or use diffusers components as you like 🔥. -->